<a href="https://colab.research.google.com/github/ougrid/my-knowledge-resource/blob/master/Tutorial_aspect_tagging_BERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!gdown 1mKLyBFIoKKWvCihQiZ6cjqx8TKYHRWFA
!gdown 1NMH2EMoK6roGUGZnpM1WByLMYuPw6Gzp
!gdown 1tCZuz2AjPTp5ok5avZAXAAkCap8GmeIL

Downloading...
From: https://drive.google.com/uc?id=1mKLyBFIoKKWvCihQiZ6cjqx8TKYHRWFA
To: /content/tutorial_asp_train.txt
100% 520k/520k [00:00<00:00, 138MB/s]
Downloading...
From: https://drive.google.com/uc?id=1NMH2EMoK6roGUGZnpM1WByLMYuPw6Gzp
To: /content/tutorial_asp_val.txt
100% 64.4k/64.4k [00:00<00:00, 52.8MB/s]
Downloading...
From: https://drive.google.com/uc?id=1tCZuz2AjPTp5ok5avZAXAAkCap8GmeIL
To: /content/tutorial_asp_test.txt
100% 62.4k/62.4k [00:00<00:00, 68.7MB/s]


In [ ]:
!pip install datasets
!pip install seqeval
!pip install transformers[torch]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 21.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 25.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.0 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling requests-2.31.0:
      Successfully uninstalled requests-2.31.0
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24

In [ ]:
import numpy as np
from datasets import load_dataset, load_metric
from transformers import AutoTokenizer, AutoModelForTokenClassification, TrainingArguments, Trainer, DataCollatorForTokenClassification


In [ ]:
def load_data(fname,tag_id=None):
  data = []
  if tag_id is None:
    tag_id = {}
  with open(fname) as fp:
      lines = fp.readlines()
      tmp_sen = []
      for line in lines:

          if line.strip()=="":
              if len(tmp_sen) > 0:
                  tokens = []
                  tags = []
                  for token,tag in tmp_sen:
                      tokens.append(token)
                      #if tag_id is None:
                      if tag not in tag_id.keys():
                        #print(tag)
                        tag_id[tag] = len(tag_id)

                      #print(tag_id[tag])
                      tags.append(tag_id[tag])



                  data.append({"tokens":tokens,"tags":tags})
                  tmp_sen = []

              continue

          _x = line.split('\t')

          tmp_sen.append((_x[0],_x[1].strip()))
  return data,tag_id

In [ ]:
import json

def write_jsonl(data, file_path):
    with open(file_path, 'w') as file:
        for entry in data:
            json_line = json.dumps(entry)
            file.write(json_line + '\n')

In [ ]:
train_data,tag_id = load_data('tutorial_asp_train.txt')
test_data,_ = load_data('tutorial_asp_test.txt',tag_id)
val_data,_ = load_data('tutorial_asp_val.txt',tag_id)

In [ ]:
write_jsonl(train_data,'train.json')
write_jsonl(test_data,'test.json')
write_jsonl(val_data,'val.json')

In [ ]:
task = "asp"
model_checkpoint = "Geotrend/bert-base-th-cased"
batch_size = 16

file_dict = {
  "train" : "./train.json",
  "test" : "./test.json",
  "val" : "./val.json"
}

datasets =  load_dataset(
  'json',
  data_files=file_dict
)

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Generating val split: 0 examples [00:00, ? examples/s]

In [ ]:
datasets

DatasetDict({
    train: Dataset({
        features: ['tokens', 'tags'],
        num_rows: 677
    })
    test: Dataset({
        features: ['tokens', 'tags'],
        num_rows: 84
    })
    val: Dataset({
        features: ['tokens', 'tags'],
        num_rows: 86
    })
})

In [ ]:
datasets['train'][0]

{'tokens': ['สินค้า',
  'ดี',
  'มี',
  'คุณภาพ',
  'และ',
  'มาตรฐาน',
  '_',
  'quick',
  '_',
  'silver',
  'ใส่',
  'สวย',
  'ๆ',
  'งาม',
  'ๆ',
  'สบาย',
  'ๆ',
  'แต่',
  'นาฬิกา',
  'ไม่',
  'สามารถ',
  'สู้',
  'แสง',
  'ได้',
  'เพราะฉะนั้น',
  'เน้น',
  'ใส่',
  'ใน',
  'เวลา',
  'กลางคืน',
  'ถ้า',
  'จะ',
  'ดู',
  'เวลา',
  'ก็',
  'ให้',
  'กด',
  'ไฟ',
  'ก่อน',
  'ลักษณะ',
  'ตัวเรือน',
  'สวยงาม',
  'มาก',
  'ราคา',
  'ถูก',
  'ใส่',
  'สวย',
  'ๆ',
  'งาม',
  'ๆ',
  'สบาย',
  'ๆ'],
 'tags': [0,
  1,
  1,
  1,
  2,
  0,
  1,
  1,
  1,
  1,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  0,
  1,
  1,
  1,
  1,
  1,
  1,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  3,
  4,
  4,
  4,
  5,
  6,
  3,
  4,
  4,
  4,
  4,
  4,
  4]}

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/658 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/47.7k [00:00<?, ?B/s]

In [ ]:
print(tokenizer("ผมเป็นคนหล่อมาก"))
print(tokenizer(["ผม","เป็น","คนหล่อ","มาก"]))

{'input_ids': [11, 162, 2939, 6762, 4949, 2617, 8477, 3554, 7999, 3167, 12], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}
{'input_ids': [[11, 162, 2939, 12], [11, 7439, 12], [11, 4344, 8477, 3554, 8226, 12], [11, 167, 8298, 12]], 'token_type_ids': [[0, 0, 0, 0], [0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0]], 'attention_mask': [[1, 1, 1, 1], [1, 1, 1], [1, 1, 1, 1, 1, 1], [1, 1, 1, 1]]}


In [ ]:

def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(examples["tokens"], truncation=True, is_split_into_words=True)
    label_all_tokens = True
    #print('tokenized_inputs:',tokenized_inputs)
    labels = []
    lab2index = {}
    word_idxs = []
    for i, label in enumerate(examples[f"tags"]):
        #print(label)
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        #print('x word_ids',word_ids)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            if word_idx is None:
                label_ids.append(-100)
            elif word_idx != previous_word_idx:
                label_ids.append(label[word_idx])

            else:
                label_ids.append(label[word_idx] if label_all_tokens else -100)

            previous_word_idx = word_idx
        labels.append(label_ids)
        word_idxs.append(word_ids)
    tokenized_inputs["labels"] = labels
    tokenized_inputs["word_ids"] = word_idxs
    return tokenized_inputs

In [ ]:
tokenized_datasets = datasets.map(tokenize_and_align_labels, batched=True)

Map:   0%|          | 0/677 [00:00<?, ? examples/s]

Map:   0%|          | 0/84 [00:00<?, ? examples/s]

Map:   0%|          | 0/86 [00:00<?, ? examples/s]

In [ ]:
#tokenized_datasets['train']['input_ids'][0]
print(tokenized_datasets['train']['tokens'][0])
print(tokenized_datasets['train']['input_ids'][0])
print(tokenized_datasets['train']['tags'][0])
print(tokenized_datasets['train']['labels'][0])
print(tokenized_datasets['train']['word_ids'][0])

['สินค้า', 'ดี', 'มี', 'คุณภาพ', 'และ', 'มาตรฐาน', '_', 'quick', '_', 'silver', 'ใส่', 'สวย', 'ๆ', 'งาม', 'ๆ', 'สบาย', 'ๆ', 'แต่', 'นาฬิกา', 'ไม่', 'สามารถ', 'สู้', 'แสง', 'ได้', 'เพราะฉะนั้น', 'เน้น', 'ใส่', 'ใน', 'เวลา', 'กลางคืน', 'ถ้า', 'จะ', 'ดู', 'เวลา', 'ก็', 'ให้', 'กด', 'ไฟ', 'ก่อน', 'ลักษณะ', 'ตัวเรือน', 'สวยงาม', 'มาก', 'ราคา', 'ถูก', 'ใส่', 'สวย', 'ๆ', 'งาม', 'ๆ', 'สบาย', 'ๆ']
[11, 175, 4358, 4949, 6018, 154, 3132, 167, 3132, 140, 6729, 7157, 8475, 2930, 5795, 2493, 167, 2930, 7938, 8247, 6081, 76, 579, 1053, 76, 5177, 1610, 187, 4665, 5013, 175, 4953, 3552, 189, 142, 7781, 189, 175, 6443, 5754, 189, 185, 4799, 5013, 159, 2930, 8478, 8429, 2930, 188, 2939, 5013, 175, 7781, 6663, 8472, 175, 5671, 5398, 185, 4665, 3316, 188, 7828, 184, 5795, 3920, 2930, 3894, 8466, 3894, 5782, 184, 2617, 6478, 187, 4665, 5013, 6252, 8191, 138, 7111, 3316, 4949, 8482, 2617, 156, 6018, 143, 3894, 154, 5671, 8191, 138, 8490, 187, 8477, 5398, 138, 3827, 188, 7531, 138, 8226, 2617, 171, 7437, 8322

In [ ]:
model = AutoModelForTokenClassification.from_pretrained(model_checkpoint, num_labels=len(tag_id))

pytorch_model.bin:   0%|          | 0.00/370M [00:00<?, ?B/s]

Some weights of BertForTokenClassification were not initialized from the model checkpoint at Geotrend/bert-base-th-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
args = TrainingArguments(
    f"test-{task}",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=30,
    weight_decay=0.01,
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
metric = load_metric("seqeval")

<ipython-input-18-e20ba34f8cc7>:1: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("seqeval")


The repository for seqeval contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/seqeval.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


In [ ]:
id_tag = {value: key for key, value in tag_id.items()}
id_tag

{0: 'B_PRODUCTQUALITY',
 1: 'I_PRODUCTQUALITY',
 2: 'O',
 3: 'B_APPEARANCE',
 4: 'I_APPEARANCE',
 5: 'B_PRICE',
 6: 'I_PRICE',
 7: 'B_DELIVERYTIME',
 8: 'I_DELIVERYTIME',
 9: 'B_DELIVERYSERVICE',
 10: 'I_DELIVERYSERVICE',
 11: 'B_STORESERVICE',
 12: 'I_STORESERVICE',
 13: 'B_PACKAGING',
 14: 'I_PACKAGING',
 15: 'B_IMPRESSED',
 16: 'I_IMPRESSED'}

In [ ]:
def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)
    true_predictions = [
        [id_tag[p].replace('_','-') for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [id_tag[l].replace('_','-') for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    results = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

In [ ]:
data_collator = DataCollatorForTokenClassification(tokenizer)

In [ ]:
trainer = Trainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["val"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,No log,2.041522,0.026471,0.016544,0.020362,0.347534
2,No log,1.770497,0.107662,0.102022,0.104766,0.440778
3,No log,1.645736,0.148472,0.156250,0.152262,0.486259
4,No log,1.567690,0.232558,0.220588,0.226415,0.528621
5,No log,1.535655,0.266779,0.292279,0.278947,0.525741
6,No log,1.500401,0.299758,0.341912,0.319450,0.557782
7,No log,1.588660,0.313146,0.389706,0.347256,0.518181
8,No log,1.573907,0.330428,0.383272,0.354894,0.544822
9,No log,1.665591,0.348165,0.409926,0.376530,0.542782
10,No log,1.692805,0.326797,0.413603,0.365112,0.519501


/usr/local/lib/python3.10/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


TrainOutput(global_step=1290, training_loss=0.6445260040519774, metrics={'train_runtime': 808.595, 'train_samples_per_second': 25.118, 'train_steps_per_second': 1.595, 'total_flos': 2366054710557528.0, 'train_loss': 0.6445260040519774, 'epoch': 30.0})

In [ ]:
trainer.evaluate()

{'eval_loss': 2.2225093841552734,
 'eval_precision': 0.3622222222222222,
 'eval_recall': 0.4494485294117647,
 'eval_f1': 0.4011484823625923,
 'eval_accuracy': 0.5522620904836193,
 'eval_runtime': 1.2936,
 'eval_samples_per_second': 66.481,
 'eval_steps_per_second': 4.638,
 'epoch': 30.0}

In [ ]:
tokenized_datasets["test"]

Dataset({
    features: ['tokens', 'tags', 'input_ids', 'token_type_ids', 'attention_mask', 'labels', 'word_ids'],
    num_rows: 84
})

In [ ]:
def most_frequent(List):
    return max(set(List), key = List.count)

In [ ]:
predictions, labels, _ = trainer.predict(tokenized_datasets["test"])
predictions = np.argmax(predictions, axis=2)

_predictions= []
_labels = []

for i in range(len(tokenized_datasets["test"]['tokens'])):
  l = []
  p = []
  last_pos = 0
  for j in range(len(tokenized_datasets["test"]['tags'][i])):
    wid = tokenized_datasets["test"]['word_ids'][i]
    #print(wid)
    #print(j)
    #print(wid.index(j),wid.index(j+1),predictions[i][wid.index(j):wid.index(j+1)],labels[i][wid.index(j):wid.index(j+1)])
    #print(i,j,predictions[i][j],labels[i])

    p.append(most_frequent(list(predictions[i][wid.index(j):wid.index(j)+wid.count(j)])))
    l.append(most_frequent(list(labels[i][wid.index(j):wid.index(j)+wid.count(j)])))

  _predictions.append(p)
  _labels.append(l)
  #break


true_predictions = [
    [id_tag[p].replace('_','-') for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(_predictions, _labels)
]
true_labels = [
    [id_tag[l].replace('_','-') for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(_predictions, _labels)
]

results = metric.compute(predictions=true_predictions, references=true_labels)
results

{'APPEARANCE': {'precision': 0.26865671641791045,
  'recall': 0.36,
  'f1': 0.3076923076923077,
  'number': 50},
 'DELIVERYSERVICE': {'precision': 0.38461538461538464,
  'recall': 0.3333333333333333,
  'f1': 0.3571428571428571,
  'number': 15},
 'DELIVERYTIME': {'precision': 0.3218390804597701,
  'recall': 0.509090909090909,
  'f1': 0.39436619718309857,
  'number': 55},
 'IMPRESSED': {'precision': 0.5,
  'recall': 0.06666666666666667,
  'f1': 0.11764705882352941,
  'number': 15},
 'PACKAGING': {'precision': 0.32432432432432434,
  'recall': 0.4444444444444444,
  'f1': 0.375,
  'number': 27},
 'PRICE': {'precision': 0.3157894736842105,
  'recall': 0.3870967741935484,
  'f1': 0.34782608695652173,
  'number': 31},
 'PRODUCTQUALITY': {'precision': 0.23300970873786409,
  'recall': 0.24489795918367346,
  'f1': 0.23880597014925373,
  'number': 98},
 'STORESERVICE': {'precision': 0.19444444444444445,
  'recall': 0.30434782608695654,
  'f1': 0.23728813559322037,
  'number': 23},
 'overall_precis

In [ ]:
true_predictions = [
    [id_tag[p].replace('B_','').replace('I_','')  for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(_predictions, _labels)
]
true_labels = [
    [id_tag[l].replace('B_','').replace('I_','') for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(_predictions, _labels)
]

In [ ]:
from pandas.core.common import flatten

In [ ]:
true_labels = list(flatten(true_labels))
true_predictions = list(flatten(true_predictions))

In [ ]:
import numpy as np
from sklearn.metrics import classification_report

targets = ['APPEARANCE','DELIVERYSERVICE','DELIVERYTIME','IMPRESSED','PACKAGING','PRICE','PRODUCTQUALITY','STORESERVICE']
# Print out the classification report
print(classification_report(
    true_labels,true_predictions,labels=targets))

                 precision    recall  f1-score   support

     APPEARANCE       0.39      0.45      0.42       247
DELIVERYSERVICE       0.34      0.45      0.39        64
   DELIVERYTIME       0.67      0.70      0.68       350
      IMPRESSED       1.00      0.02      0.04        88
      PACKAGING       0.69      0.61      0.65       203
          PRICE       0.78      0.44      0.56       239
 PRODUCTQUALITY       0.69      0.62      0.65       588
   STORESERVICE       0.48      0.46      0.47       186

      micro avg       0.60      0.54      0.57      1965
      macro avg       0.63      0.47      0.48      1965
   weighted avg       0.64      0.54      0.56      1965



In [ ]:
                 precision    recall  f1-score   support

     APPEARANCE       0.50      0.34      0.40       247
DELIVERYSERVICE       0.30      0.39      0.34        64
   DELIVERYTIME       0.68      0.68      0.68       350
      IMPRESSED       0.00      0.00      0.00        88
      PACKAGING       0.77      0.55      0.64       203
          PRICE       0.76      0.46      0.57       239
 PRODUCTQUALITY       0.68      0.55      0.61       588
   STORESERVICE       0.64      0.39      0.49       186

      micro avg       0.65      0.49      0.56      1965
      macro avg       0.54      0.42      0.47      1965
   weighted avg       0.63      0.49      0.55      1965

Ref : https://github.com/huggingface/notebooks/blob/main/examples/token_classification.ipynb